In [ ]:
np.random.seed(42)

In [ ]:
##utilis
import os
import math
import time
from time import strftime
import numpy as np
import numpy.linalg as LA
import scipy.io as sio
import pkg_resources


def repmat(A, rows, cols):
    return np.tile(A, (cols, rows)).T


def vec(A):
    # TODO: rewrite docstrings
    """
    * Syntax: `a = vec(A)`
    * Vectorization of a matrix. This function is a built-in function in some
    recent MATLAB version.
    """
    return A.flatten()


def label_to_range(label):
    """
    Convert label to range

    Parameters:
    -----------
    label: list of integers
        must be in the form of [1, 1, ..., 1, 2, 2, ..., 2, ..., C, C, ..., C]
        i.e. nondecreasing numbers starting from 1, each element is greater
        than the previous element by at most 1

    Returns:
    --------
    a list of intergers with C + 1 elements, start with 0
    the i-th element is number of elements in label that equals to i

    """
    res = [0]
    # assert label[0] == 1, 'label must start with 1'
    for i in range(1, len(label)):
        if label[i] == label[i-1]:
            continue
        if label[i] == label[i-1] + 1:
            res.append(i)
        else:
            assert False,\
                ('label[{}] and label[{}] must be equal or two consecutive '
                 'integers, got {} and {}').format(
                     i-1, i, label[i-1], label[i]
                 )
    res.append(len(label))
    return res


def range_to_label(a_range):
    """
    From a range, convert it to label

    This is an inverse function of label_to_range
    Parameters:
    -----------
    a_range: list of integers
        must start with 0 and is a strictly increasing list

    Returns:
    --------

    """
    assert a_range[0] == 0, 'input must start with 0'
    res = []
    for i in range(1, len(a_range)):
        assert a_range[i] > a_range[i-1],\
            ('a_range must be an increasing list, '
             'got a_range[{}] = {} < a_range[{}] = {}').format(
                 i, a_range[i], i - 1, a_range[i-1]
             )

        res.extend([i]*(a_range[i] - a_range[i-1]))
    return res


def get_block_row(matrix, block_indices, row_range):
    """
    Extract a subset of rows from a matrix

    Parameters:
    -----------
    matrix: 2-d numpy array
        block matrix
    block_indices: integer of list of integers
        indices of extracted blocks, 0-indexed. If indices is a list, return
        the concatenation of all blocks
    row_range: list of intergers
        in the form of [0, c_1, c_1 + c_2, ..., c_1 + c_2 + ... + c_N]
        where c_i is the number of rows in the i-th block

    Returns:
    --------
    a 2-d matrix
    """
    assert matrix.ndim == 2, 'Expect to receive 2-d array input, got shape {}'.format(matrix.shape)
    if isinstance(block_indices, int):
        block_indices = [block_indices]
    # if isinstance(block_indices, (list, np.ndarray, np.generic))
    ids = []
    for i in block_indices:
        ids = ids + list(range(row_range[i], row_range[i+1]))
    return matrix[ids, :].copy()


def get_block_col(matrix, block_indices, col_range):
    """
    Extract a subset of columns from a matrix

    Parameters:
    -----------
    matrix: 2-d numpy array
        block matrix
    block_indices: integer of list of integers
        indices of extracted blocks, 1-indexed. If indices is a list, return
        the concatenation of all blocks
    row_range: list of intergers
        in the form of [0, c_1, c_1 + c_2, ..., c_1 + c_2 + ... + c_N]
        where c_i is the number of columns in the i-th block

    Returns:
    --------
    a 2-d matrix
    """
    assert matrix.ndim == 2, 'Expect to receive 2-d array input, got shape {}'.format(matrix.shape)
    # assert matrix.shape[1] == col_range[-1]
    return get_block_row(matrix.T, block_indices, col_range).T


def get_block(matrix, i, j, row_range, col_range):
    """
    Extract a submatrix of a matrix

    Parameters:
    -----------
    matrix the big matrix:
    matrix = [ M11, M12, ..., M1m;
               M21, M22, ..., M2m;
               ... ;
               Mn1, Mn2, ..., Mnm]
    i: row block index
    j: column block index
    row_range: row range
    col_range: columns range
    """
    return matrix[row_range[i]:row_range[i+1],
                  col_range[j]: col_range[j+1]].copy()


def norm1(X):
    """
    Return norm 1 of a matrix, which is sum of the absolute value of all elements
    of that matrix.
    """
    if X.shape[0]*X.shape[1] == 0:
        return 0
    return abs(X).sum()


def normF2(X):
    """
    Return square of the Frobenius norm, which is sum of square of all
    elements in a matrix
    """
    if X.shape[0]*X.shape[1] == 0:
        return 0
    return LA.norm(X, 'fro')**2


def normc(A):
    """
    normalize each column of A to have norm2 = 1
    """
    return A / np.tile(np.sqrt(np.sum(A*A, axis=0)), (A.shape[0], 1))


def nuclearnorm(X):
    """
    Return nuclear norm of a matrix.
    """
    if X.size == 0:
        return 0
    return LA.norm(X) if X.ndim == 1 else LA.norm(X, 'nuc')


def shrinkage(U, alambda):
    """
    Soft thresholding function.

    Solve the following optimization problem:
    X = arg min_X 0.5*||X - U||_F^2 + lambda||X||_1
    where U and X are matrices with same sizes. lambda can be either a positive
    scalar or a positive matrix (all elements are positive) with same size as X.
    """
    return np.maximum(0, U - alambda) + np.minimum(0, U + alambda)


def shrinkage_rank(D, alambda):
    """
    Singular value thresholding algorithm for matrix completion.
    Solve the following optimization problem:
      X = arg min_X 0.5*||X - D||_F^2 + lambda*||X||_*
      where ||X||_* is the nuclear norm.
    """
    U, s, V = LA.svd(D, full_matrices=False)
    s1 = np.maximum(0, s - alambda)
    return np.dot(U, np.dot(np.diag(s1), V))


class MyForm:
    """
    Describe a special family of matrices:
    A = [   M 0 ... 0;
            0 M ... 0;
            0 0 ... M] +
        [   N N ... N;
            N N ... N;
            N N ... N]
    with k block rows and columns
    """

    def __init__(self, M, N, k):
        self.M = M.copy()
        self.N = N.copy()
        self.k = k

    def full_express(self):
        return np.kron(np.eye(self.k), self.M) + np.tile(self.N, (self.k, self.k))

    def mult(self, other):
        """
        Multiplication with other MyForm matrix
        """
        A = np.dot(self.M, other.M)
        B = np.dot(self.M, other.N) + np.dot(self.N, other.M) + \
            self.k*np.dot(self.N, other.N)
        return MyForm(A, B, self.k)

    def inv(self):
        """
        compute inverse matrix
        """
        A = LA.inv(self.M)
        B = - np.dot(LA.inv(self.M + self.k*self.N), np.dot(self.N, A))
        return MyForm(A, B, self.k)

    def mult_vec(self, x):
        """
        return M*x (matrix vector multiplication)
        """
        X = x.reshape(self.M.shape[1], self.k, order='F')
        p = np.dot(self.N, X).sum(axis=1)

        return vec(np.dot(self.M, X)) + vec(np.tile(p, (1, self.k)))


def randperm(n):
    """
    get a random permutation of range(n)
    """
    return np.random.permutation(list(range(n)))


# def get_range(arange, c):
#     return list(range(arange[c], arange[c+1]))


def pickDfromY(Y, Y_range, D_range):
    """
    randomly pick k_c samples from Y_c
    """
    C = len(Y_range) - 1
    D = np.zeros((Y.shape[0], D_range[-1]))
    for c in range(C):
        Yc = get_block_col(Y, c, Y_range)
        N_c = Yc.shape[1]
        # print Yc
        ids = randperm(N_c)
        kc = D_range[c+1] - D_range[c]
        D[:, D_range[c]:D_range[c+1]] = Yc[:, np.sort(ids[:kc])]
    return D


def load_mat(filename):
    return sio.loadmat(filename)


def picl_train_test(dataset, N_train_c):
    # data_fn = pkg_resources.resource_filename('dictol', 'data/'+dataset + '.mat')
    vars_dict = load_mat("myYaleB.mat")
    Y = vars_dict['Y']
    d = Y.shape[0]
    if 'Y_range' not in vars_dict:
        Y_range = label_to_range(vars_dict['label'].flatten()).astype(int)

    else:
        Y_range = vars_dict['Y_range'].flatten().astype(int)
    print(Y_range)


    C = len(Y_range) - 1
    N_total     = Y_range[-1]
    N_train     = C*N_train_c
    N_test      = N_total - N_train

    Y_train     = np.zeros((d, N_train))
    Y_test      = np.zeros((d, N_test))
    label_train = [0]*N_train
    label_test = [0]*N_test
    cur_train   = 0
    cur_test    = 0
    for c in range(C):
        Yc        = get_block_col(Y, c, Y_range)
        N_total_c = Yc.shape[1]
        N_test_c  = N_total_c - N_train_c
        label_train[cur_train: cur_train + N_train_c] = [c+1]*N_train_c
        label_test[cur_test:cur_test + N_test_c] = [c+1]*N_test_c

        ids = randperm(N_total_c)

        Y_train[:, cur_train: cur_train + N_train_c] = \
            Yc[:, np.sort(ids[:N_train_c])]

        Y_test[:, cur_test: cur_test + N_test_c] = \
            Yc[:, np.sort(ids[N_train_c:])]

        cur_train += N_train_c
        cur_test += N_test_c

    Y_train = normc(Y_train)
    Y_test  = normc(Y_test)
    return (Y_train, label_train, Y_test, label_test)


def range_reduce(D_range, bad_ids):
    C = D_range.size - 1
    for c in range(C):
        cumk = D_range[c+1]
        e = cumk - np.nonzero(bad_ids < cumk)[0].size
        D_range[c+1] = e


# range_reduce_test()
def build_mean_vector(X, Y_range):
    """
    M = build_mean_vector(X, Y_range)
    suppose X = [X_1 X_2 ... X_C]
    return M = [m1, m2, ..., M_C]
    where mi = mean(X_i)
    """
    C = Y_range.size -1
    M = np.zeros((X.shape[0], C))
    for c in range(C):
        Xc = get_block_col(X, c, Y_range)
        M[:, c] = np.mean(Xc, axis=1)
    return M

from sklearn.model_selection import train_test_split

def training_test_split(dataset, N_train):
    if dataset == 'myARgender':
        fn = pkg_resources.resource_filename('dictol', 'data/'+dataset + '.mat')
        # fn = os.path.join('data', 'myARgender.pickle')
        vars_dict = load_mat(fn)
        Y_train = vars_dict['Y_train']
        Y_test = vars_dict['Y_test']
        label_train = vec(vars_dict['label_train']).astype(int)

        label_test = vec(vars_dict['label_test']).astype(int)
        range_train = label_to_range(label_train)
        # range_test  = label_to_range(label_test)

        new_range_train = N_train * np.arange(N_train + 1)
        Y_train         = pickDfromY(Y_train, range_train, new_range_train)
        label_train     = range_to_label(new_range_train)

        Y_train = normc(Y_train)
        Y_test  = normc(Y_test)

    elif dataset == 'myARreduce':
        fn = os.path.join('data', 'AR_EigenFace.pickle')
        vars_dict = load_mat(fn)

        Y_train     = normc(vars_dict['tr_dat'])
        Y_test      = normc(vars_dict['tt_dat'])
        label_train = vec(vars_dict['trls']).astype(int)
        label_test  = vec(vars_dict['ttls']).astype(int)

    elif dataset == 'myFlower':
        dataset = 'myFlower102'
        # fn      = os.path.join('data', dataset + '.pickle')
        vars_dict    = load_mat("myFlower102.mat")

        Y_train     = vars_dict['Y_train']
        Y_test      = vars_dict['Y_test']
        label_train = vec(vars_dict['label_train'])
        label_test  = vec(vars_dict['label_test'])
        # print("label train", label_train[52000])
        range_train = label_to_range(label_train)
        print("range", range_train)
        num_classes = len(range_train) - 1
        new_range_train = N_train * np.arange(num_classes + 1)
        label_train = range_to_label(new_range_train)
        Y_train = pickDfromY(Y_train, range_train, new_range_train)

        Y_train = normc(Y_train)
        Y_test = normc(Y_test)

    elif dataset == 'mnist':
        # dataset = 'myFlower102'
        # fn      = os.path.join('data', dataset + '.pickle')
        vars_dict    = load_mat("mnist-original.mat")
        mnist_data = vars_dict["data"].T
        # mnist_data = mnist_data.reshape(len(mnist_data),28,28,1)
        mnist_label = vars_dict["label"][0]
        # Y_train, Y_test, label_train, label_test = [],[],[],[]
        # for i in range(10):
        #   Y_train.append(mnist_data[i*7000:(i+1)*7000 - 1000])
        #   Y_test.append(mnist_data[(i+1)*7000 - 1000:(i+1)*7000])
        #   label_train.append(mnist_label[i*7000:(i+1)*7000 - 1000])
        #   label_test.append(mnist_label[(i+1)*7000 - 1000:(i+1)*7000])

        # Y_train = np.array(Y_train)
        # Y_test = np.array(Y_test)
        # label_train = np.array(label_train).flatten()
        # label_test = np.array(label_test).flatten()

        Y_train, Y_test, label_train, label_test = train_test_split(mnist_data,mnist_label,train_size=60000,test_size=10000,shuffle=False)
        print(Y_train.shape)
        print(Y_test.shape)
        Y_train     = Y_train.T
        Y_test      = Y_test.T
        print("label train", label_train.shape)
        label_train = vec(label_train)
        label_test  = vec(label_test)
        range_train = label_to_range(label_train)
        range_test  = label_to_range(label_test)

        print("label train", label_train)
        print("range", range_train)
        num_classes = len(range_train) - 1
        new_range_train = N_train * np.arange(num_classes + 1)
        new_range_test = N_train * np.arange(num_classes + 1)
        label_train = range_to_label(new_range_train)
        label_test = range_to_label(new_range_test)
        Y_train = pickDfromY(Y_train, range_train, new_range_train)
        Y_test = pickDfromY(Y_test, range_test, new_range_test)
        print("Y_train",Y_train)
        print("Label_train",label_train)
        print("Y_test",Y_test)
        print("Label_test",label_test)
        Y_train = normc(Y_train)
        Y_test = normc(Y_test)
    else:
        Y_train, label_train, Y_test, label_test = picl_train_test(dataset, N_train)
    return (Y_train, Y_test, label_train, label_test)


#################### DLCOPAR #######################
def buildMhat(M, range_row, range_col):
    """
    buildMhat(M, range_row, range_col):
    suppose M = [M11 M12 ... M1n;
                      M21 M22 ... M3n;
                      .....
                      Mn1 Mn2 .... Mnn]
        then Mhat = = [2*M11  M12     ... M1n;
                       M21    2*M22   ... M3n;
                          .....
                       Mn1     Mn2 .... 2*Mnn]
    ---------------------------------------------
    Author: Tiep Vu, thv102@psu.edu, 04/21/2016
            http://www.personal.psu.edu/thv102/
    ---------------------------------------------
    """
    C = len(range_row) - 1
    M2 = M.copy()
    for c in range(C):
        M2[range_row[c]: range_row[c+1], range_col[c]: range_col[c+1]] *= 2
    return M2


def buildM_2Mbar(X, Y_range, lambda2):
    """
    """
    MM = np.zeros_like(X)
    C = len(Y_range) - 1
    m = np.mean(X, axis=1)
    for c in range(C):
        Xc = get_block_col(X, c, Y_range)
        mc = np.mean(Xc, axis=1)
        MM[:, Y_range[c]: Y_range[c+1]] = repmat(lambda2*(m - 2*mc), 1, Y_range[c+1] - Y_range[c])
    return MM


def build_mean_matrix(X, cols=None):
    """
    repeat np.mean(X, axis = 1) cols times.

    ---------------
    Parameters:
    X: 2d numpy array
    cols: int
        if cols == None then cols = #cols of X

    """
    if len(X.shape) < 2 or X.shape[1] == 0:
        return X
    mean_vector = np.mean(X, axis=1)
    if cols is None:
        return repmat(mean_vector, 1, X.shape[1])
    else:
        return np.tile(mean_vector, cols)


def range_delete_ids(a_range, ids):
    """
    given a range a_range of an array. Suppose we want to delete some
    element of that array indexed by `ids`, `new_range` is the new range
    """
    ids = np.sort(ids)
    n = a_range.size
    a = np.zeros_like(a_range)
    j = 1
    while j < n-1:
        for i in range(n):
            while a_range[j] < ids[i]:
                j += 1
            for k in range(j, n):
                a[k] += 1

    new_range = a_range - a
    return new_range


def range_delete_ids_test():
    a_range = np.array([0, 3, 5, 10])
    ids = np.array([1, 4, 7, 10])
    print(a_range)
    print(range_delete_ids(a_range, ids))


def max_eig(D):
    """
    return maximum eigenvalue of matrix D
    """
    return np.max(LA.eig(D)[0])


def erase_diagonal(A):
    if A.shape[0] != A.shape[1]:
        print('The input matrix is not square!')
        return
    B = A.copy()
    np.fill_diagonal(B, 0)
    return B


def erase_diagonal_blocks(A, row_range, col_range):
    """ remove diagonal blocks of the block matrix A """
    if len(row_range) != len(col_range):
        print('no. of column blocks != no. of row blocks!!')
    C = len(row_range) - 1
    B = A.copy()
    for c in range(C):
        B[row_range[c]: row_range[c+1], col_range[c]: col_range[c+1]] = 0
    return B


def inv_IpXY(X, Y):
    """
    Calculate the inverse of matrix A = I + XY.
    if X is a fat matrix (number of columns >> number of rows), then use inv(I + X*Y)
    else: use equation: (I + XY)^(-1) = I - Y*(I + Y*X)^(-1)*X
    """
    d1 = X.shape[0]
    d2 = X.shape[1]
    if d1 > d2:
        M = np.eye(d1) - np.dot(np.dot(X, LA.inv(np.eye(d2) + np.dot(Y, X))), Y)
    else:
        M = LA.inv(np.eye(d1) + np.dot(X, Y))
    return M


def progress_str(cur_val, max_val, total_point=50):
    p = int(math.ceil(float(cur_val)*total_point / max_val))
    return '|' + p*'#' + (total_point - p)*'.' + '|'


def get_time_str():
    print('Time now: ' + strftime("%m/%d/%Y %H:%M:%S"))
    return strftime("%m%d_%H%M%S")


In [ ]:
###optimise
def ODL_updateD(D, E, F, iterations=100, tol=1e-8):
    """
    The main algorithm in ODL.
    Solving the optimization problem:
      D = arg min_D -2trace(E'*D) + trace(D*F*D') subject to: ||d_i||_2 <= 1,
         where F is a positive semidefinite matrix.

    Parameters:
    -----------
    D, E, F as in the above problem.
    iterations: maximum number of iterations.
    tol: when the difference of solutions in two successive
        iterations less than this value, the algorithm will stop.

    Returns:
    --------
    """
    def calc_cost(D):
        return -2*np.trace(np.dot(E, D.T)) + np.trace(np.dot(np.dot(F, D.T), D))

    D_old = D.copy()
    for _ in range(iterations):
        for i in range(D.shape[1]):
            if F[i, i] != 0:
                a = 1.0/F[i, i] * (E[:, i] - D.dot(F[:, i])) + D[:, i]
                D[:, i] = a/max(LA.norm(a, 2), 1)

        if LA.norm(D - D_old, 'fro')/D.size < tol:
            break
        D_old = D.copy()
    return D


def DLSI_updateD(D, E, F, A, lambda1, iterations=100):
    """
    def DLSI_updateD(D, E, F, A, lambda1, verbose = False, iterations = 100):
    problem: `D = argmin_D -2trace(ED') + trace(FD'*D) + lambda *||A*D||F^2,`
    subject to: `||d_i||_2^2 <= 1`
    where F is a positive semidefinite matrix
    ========= aproach: ADMM ==============================
    rewrite: `[D, Z] = argmin -2trace(ED') + trace(FD'*D) + lambda ||A*Z||_F^2,`
        subject to `D = Z; ||d_i||_2^2 <= 1`
    aproach 1: ADMM.
    1. D = -2trace(ED') + trace(FD'*D) + rho/2 ||D - Z + U||_F^2,
        s.t. ||d_i||_2^2 <= 1
    2. Z = argmin lambda*||A*Z|| + rho/2||D - Z + U||_F^2
    3. U = U + D - Z
    solve 1: D = argmin -2trace(ED') + trace(FD'*D) + rho/2 ||D - W||_F^2
                          with W = Z - U;
               = argmin -2trace((E - rho/2*W)*D') +
                  trace((F + rho/2 * eye())*D'D)
    solve 2: derivetaive: 0 = 2A'AZ + rho (Z - V) with V = D + U
    `Z = B*rhoV` with `B = (2*lambda*A'*A + rho I)^{-1}`
    `U = U + D - Z`
    -----------------------------------------------
    """
    def calc_cost(D):
        cost = -2*np.trace(np.dot(E, D.T)) + np.trace(np.dot(F, np.dot(D.T, D))) +\
            lambda1*normF2(np.dot(A, D))
        return cost
    it = 0
    rho = 1.0
    Z_old = D.copy()
    U = np.zeros_like(D)
    I_k = np.eye(D.shape[1])
    X = 2*lambda1/rho*A.T
    Y = A.copy()
    B1 = np.dot(X, inv_IpXY(Y, X))

    # B1 = np.dot(X, LA.inv(eye(Y.shape[0]) + np.dot(Y, X)))
    tol = 1e-8
    for it in range(iterations):
        it += 1
        # update D
        W  = Z_old - U
        E2 = E + rho/2*W
        F2 = F + rho/2*I_k
        D  = ODL_updateD(D, E2, F2)
        # update Z
        V = D + U
        Z_new = rho*(V - np.dot(B1, np.dot(Y, V)))
        e1 = normF2(D - Z_new)
        e2 = rho*normF2(Z_new - Z_old)
        if e1 < tol and e2 < tol:
            break
        U = U + D - Z_new
        Z_old = Z_new.copy()

    return D


def num_grad(func, X):
    """
    Calculating gradient of a function `func(X)` where `X` is a matrix or
    vector
    """
    grad = np.zeros_like(X)
    eps = 1e-4
    # TODO: flatten then unflatten, make it independent on X.shape
    # the current implementation only work with 2-d array
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # print X, '\n'
            Xp = X.copy()
            Xm = X.copy()
            Xp[i, j] += eps
            # print X
            fp = func(Xp)
            Xm[i, j] -= eps
            fm = func(Xm)
            grad[i, j] = (fp - fm)/(2*eps)
    return grad


def check_grad(func, grad, X):
    print('Checking grad...',)
    grad1 = grad(X)
    grad2 = num_grad(func, X)

    dif =  LA.norm(grad1 - grad2)
    if dif < 1e-5:
        print('Different = %f' %dif, 'PASS')
    else:
        print('Different = %f' %dif, 'FAIL')
    return dif < 1e-5


def min_rank_dict(Y, X, lambdaD, Dinit, iterations = 100, tol = 1e-8):
    """
    [D] = argmin_D 0.5*|| Y - DX||_F^2 + lambdaD ||D||_*
    s.t. ||d_i||_2^2 <= 1, for all i
    using ADMM:
    INPUT:
        Y: Data
        Dinit: intitial D
        X: sparse code
        lambdaD: regularization term
    OUTPUT:
        D:
    Created: Tiep Vu 6/29/2015 2:05:28 PM
    ------------------------
    Choose a rho.
    Algorithm summary
    ADMM: D,J = argmin_DJ 0.5*||Y - DX||_F^2 + lambdaD||J||_*
    s.t ||d_i||_2^2 <= 1 and J = D
    Alternatively solving:
    (1): D^{k+1} = argmin_D 0.5*||Y - DX||_F^2 + rho/2 ||J - D + U^k||_F^2
        s.t. ||d_i||_2^2 <= 1
        this problem can be soved using the update dictionary stage in
            Online Dictionary Learning method
    (2): J^{k+1} = argminJ lambdaD||J||_* + rho/2||J - D^{k+1} + U^k||
        Solution: shrinkage_rank(D^{k+1} - U^k, lambdaD/rho)
    (3): Update U: U^{k+1} = U^k + J^{k+1} - D^{k+1}
     Stoping cretia:
    ||r^k||_F^2 <= tol, ||s^k||_F^2 <= tol
    r^k = J^k - D^k
    s^k = rho(J^{k+1} - J^k)
    ---------------------------------------------
    Author: Tiep Vu, thv102@psu.edu, 04/22/2016
            http://www.personal.psu.edu/thv102/
    ---------------------------------------------
    """
    YXt = np.dot(Y, X.T)
    XXt = np.dot(X, X.T)
    rho = 0.25
    D_old = Dinit
    J_old = Dinit
    U_old = np.zeros_like(Dinit)
    it = 0
    I = np.eye(XXt.shape[0])
    tau = 2
    mu = 10.0
    for it in range(iterations):
        ## =========update D ================================
        # D = argmin_D 0.5*||Y - DX||_F^2 + rho/2 ||J - D + U||_F^2
        # s.t. ||d_i||_2^2 <= 1
        E = YXt + rho*(J_old + U_old)
        F = XXt + rho*I
        # D_new = updateD_EF(D_old, E, F, 10);
        D_new = ODL_updateD(D_old, E, F, iterations = 30)
        ## ========= update J ==============================
        # J^{k+1} = argminJ lambdaD||J||_* + rho/2||J - D + U||
        J_new = np.real(shrinkage_rank(D_old - U_old, lambdaD/rho))
         ## ========= update U ==============================
        U_new = U_old + J_new - D_old
        ## ========= check stop ==============================
        r = J_new - D_old
        s = rho*(J_new - J_old)
        r_eps = LA.norm(r, 'fro')
        s_eps = LA.norm(s, 'fro')
        if r_eps < tol and s_eps < tol:
            break
        D_old = D_new
        J_old = J_new
        U_old = U_new
        if r_eps > mu*s_eps:
            rho = rho*tau
        elif s_eps > mu*r_eps:
            rho = rho/tau
    return D_new


class Fista(object):
    def __init__(self):
        """
        subclasses are required to have three following functions and lambd
        """
        self._grad = None
        self._calc_f = None
        self.lossF = None
        self.lambd = None
        self.D = None
        self.Y = None
        self.L = None

    def solve(self, Xinit=None, iterations=100, tol=1e-8, verbose=False):
        if Xinit is None:
            Xinit = np.zeros((self.D.shape[1], self.Y.shape[1]))
        Linv = 1/self.L
        lambdaLiv = self.lambd/self.L
        x_old = Xinit.copy()
        y_old = Xinit.copy()
        t_old = 1
        it = 0
        # cost_old = float("inf")
        for it in range(iterations):
            x_new = np.real(shrinkage(y_old - Linv*self._grad(y_old), lambdaLiv))
            t_new = .5*(1 + math.sqrt(1 + 4*t_old**2))
            y_new = x_new + (t_old - 1)/t_new * (x_new - x_old)
            e = norm1(x_new - x_old)/x_new.size
            if e < tol:
                break
            x_old = x_new.copy()
            t_old = t_new
            y_old = y_new.copy()
            if verbose:
                print('iter \t%d/%d, loss \t %4.4f'%(it + 1, iterations, self.lossF(x_new)))
        return x_new

    def _grad(self, y):
        raise NotImplementedError

    def lossF(self, x):
        raise NotImplementedError

    def check_grad(self, X):
        grad1 = self._grad(X)
        grad2 = num_grad(self._calc_f, X)
        dif = norm1(grad1 - grad2)/grad1.size
        print('grad difference = %.7f'%dif)


class Lasso(Fista):
    """
    Solving a Lasso optimization problem using FISTA
    `X, = arg min_X 0.5*||Y - DX||_F^2 + lambd||X||_1
        = argmin_X f(X) + lambd||X||_1
        F(x) = f(X) + lamb||X||_1
    """
    def __init__(self, D, lambd = .1):
        self.D = D
        self.lambd = lambd
        self.DtD = np.dot(self.D.T, self.D)
        self.Y = None
        self.DtY = None
        self.L = np.max(LA.eig(self.DtD)[0])
        self.coef_ = None

    def fit(self, Y, Xinit = None, iterations = 100):
        self.Y = Y
        self.DtY = np.dot(self.D.T, self.Y)
        if Xinit is None:
            Xinit = np.zeros((self.D.shape[1], self.Y.shape[1]))
        self.coef_ = self.solve(Xinit=Xinit, iterations=iterations)

    def _grad(self, X):
        return np.dot(self.DtD, X) - self.DtY

    def _calc_f(self, X):
        return 0.5*normF2(self.Y - np.dot(self.D, X))

    def lossF(self, X):
        return self._calc_f(X) + self.lambd*norm1(X)

In [ ]:
### Odl
class ODL(object):
    """
    Solving the optimization problem:
        (D, X) = arg min_{D, X} 0.5||Y - DX||_F^2 + lamb||X||_1
    """
    def __init__(self, k, lambd=0.001, updateD_iters=100, updateX_iters=100):
        self.lambd = lambd
        self.k = k
        self.Y = None
        self.D = None
        self.X = None
        self.updateD_iters = updateD_iters
        self.updateX_iters = updateX_iters

    def fit(self, Y, iterations=100, verbose=False):
        """
        Y: numpy data [n_features, n_samples]
        k: interger: number of atoms in the dictionary
            if k is None, select k = round(0.2*n_samples)
        """
        self.Y = Y
        Y_range = np.array([0, self.Y.shape[1]])
        D_range = np.array([0, self.k])
        self.D = pickDfromY(self.Y, Y_range, D_range)
        self.X = np.zeros((self.D.shape[1], self.Y.shape[1]))
        for it in range(iterations):
            # update X
            lasso = Lasso(self.D, self.lambd)
            lasso.fit(self.Y, Xinit = self.X)
            self.X = lasso.coef_
            # update D
            F = np.dot(self.X, self.X.T)
            E = np.dot(self.Y, self.X.T)
            self.D = ODL_updateD(self.D, E, F, iterations = self.updateD_iters)
            if verbose:
                print('iter \t%d/%d \t\t loss \t%.4f'%(it, iterations, self.loss()))

    def loss(self):
        loss = 0.5*normF2(self.Y - np.dot(self.D, self.X)) + \
                self.lambd*norm1(self.X)
        return loss


In [ ]:
### base
class BaseModel(object):
    """
    base dictionary learning model for classification
    """
    # def __init__(self)
    def predict(self, Y):
        N = Y.shape[1]
        E = np.zeros((self.num_classes, N))
        for c in range(self.num_classes):
            # Dc in D only
            Dc_ = get_block_col(self.D, c, self.D_range)
            # Dc in D and D0
            Dc = np.hstack((Dc_, self.D0)) if self.k0 > 0 else Dc_
            lasso = Lasso(Dc, lambd=self.lambd)
            lasso.fit(Y)
            Xc = lasso.solve()
            residual_matrix = Y - np.dot(Dc, Xc)
            E[c, :] = 0.5*np.sum(residual_matrix*residual_matrix, axis=0) +\
                self.lambd*np.sum(np.abs(Xc), axis=0)
        pred = np.argmin(E, axis=0) + 1
        return pred


    def evaluate(self, data, label):
        pred = self.predict(data)
        acc = np.sum(pred == label)/float(len(label))
        print('accuracy = {:.2f} %'.format(100 * acc))
        return acc


In [ ]:
import numpy as np

In [ ]:
"""
Low-rank Shared Dictionary Learning
and Fisher Discriminant Dictionary Learning
"""

# from __future__ import print_function
# from . import optimize
# from . import utils, base
# from .utils import normF2, norm1, get_block_col, get_block_row, nuclearnorm, build_mean_matrix
# import numpy as np
# from .ODL import ODL

_zero = np.array([0])

class _UpdateXX0(Fista):
    """
    solve XX0 in LRSDL using Fista
    """
    def __init__(self, Y, Y_range, D, D_range, D0, k0, lambd=0.01, lambd2=0.01):
        self.Y = Y
        self.Y_range = Y_range
        self.num_classes = len(Y_range) - 1
        self.D = D
        self.D_range = D_range
        self.D0 = D0
        self.k0 = k0
        self.lambd = lambd
        self.lambd2 = lambd2
        self.DtD = np.dot(D.T, D)
        self.D_0 = buildMhat(self.DtD, self.D_range, self.D_range)
        self.Dhat = self.D_0 + 2*self.lambd2*np.eye(self.D_0.shape[1])
        self.D0tD0 = np.dot(self.D0.T, self.D0) if self.k0 > 0 else _zero
        self.A = 2*self.D0tD0 + self.lambd2*np.eye(self.k0) if self.k0 > 0 else _zero
        self.DtD0 = np.dot(D.T, D0) if self.k0 > 0 else _zero
        self.D0tD = self.DtD0.T
        self.D0tY2 = 2*np.dot(self.D0.T, self.Y) if self.k0 > 0 else _zero

        self.DtY0 = np.dot(self.D.T, self.Y)
        self.L = max_eig(self.Dhat) + 4*self.lambd2 + 1
        if self.k0 > 0:
            self.L += max_eig(self.A)

    def _extract_fromX1(self, X1):
        K = self.D_range[-1]
        return (X1[:K, :], X1[K:, :]) if self.k0 > 0 else (X1[:K, :], _zero)

    def _grad(self, X1):
        X, X0 = self._extract_fromX1(X1)
        DtY = self.DtY0 - np.dot(self.DtD0, X0)
        Y_0 = buildMhat(DtY, self.D_range, self.Y_range)
        g = np.dot(self.Dhat, X) - Y_0 + buildM_2Mbar(X, self.Y_range, self.lambd2)
        if self.k0 > 0:
            g0 = np.dot(self.A, X0) - self.D0tY2 +\
                np.dot(self.D0tD, buildMhat(X, self.D_range, self.Y_range)) -\
                self.lambd2*build_mean_matrix(X0)
            return np.vstack((g, g0))
        return g

    def _fidelity(self, X):
        """
        * Calculating the fidelity term in FDDL[[4]](#fn_fdd):
        * $\sum_{c=1}^C \Big(\|Y_c - D_cX^c_c\|_F^2 +
            \sum_{i \neq c} \|D_c X^c_i\|_F^2\Big)$
        """
        cost = 0
        Y = self.Y
        for c in range(self.num_classes):
            Yc = get_block_col(Y, c, self.Y_range)
            Dc = get_block_col(self.D, c, self.D_range)
            Xc = get_block_row(X, c, self.D_range)
            Xcc = get_block_col(Xc, c, self.Y_range)
            cost += normF2(Yc - np.dot(Dc, Xcc))
            for i in range(self.num_classes):
                if i == c:
                    continue
                Xci = get_block_col(Xc, i, self.Y_range)
                cost += normF2(np.dot(Dc, Xci))
        return cost

    def _discriminative(self, X):
        """
        * calculating the discriminative term in
        * $\|X\|_F^2 + \sum_{c=1}^C (\|Xc - Mc\|_F^2 - \|Mc - M\|_F^2) $
        """
        cost = normF2(X)
        m = np.mean(X, axis=1)
        for c in range(self.num_classes):
            Xc = get_block_col(X, c, self.Y_range)
            Mc = build_mean_matrix(Xc)
            cost += normF2(Xc - Mc)
            M = repmat(m, 1, Xc.shape[1])
            cost -= normF2(Mc - M)
        return cost

    def _calc_f(self, X1):
        X, X0 = self._extract_fromX1(X1)
        Ybar = self.Y - np.dot(self.D0, X0)
        cost = 0.5*(normF2(Ybar - np.dot(self.D, X)) + self._fidelity(X)) + \
            0.5*self.lambd2*self._discriminative(X) + normF2(X0 - build_mean_matrix(X0))
        return cost

    def lossF(self, X1):
        return self._calc_f(X1) + self.lambd*norm1(X1)


class LRSDL(BaseModel):
    def __init__(self, lambd=0.01, lambd2=0.01, eta=0.0001,
            k=10, k0=5, updateX_iters=100, updateD_iters=100):
        self.lambd = lambd
        self.lambd2 = lambd2
        self.eta = eta
        self.D = None
        self.X = None
        self.Y = None
        self.k = k
        self.k0 = k0
        self.updateX_iters = updateX_iters
        self.updateD_iters = updateD_iters
        self.D_range = None
        self.D0 = None
        self.Y_range = None
        self.X = None
        self.X0 = None

    def _getYc(self, c):
        return get_block_col(self.Y, c, self.Y_range)

    def fit(self, Y, train_label, verbose=False, iterations=100, show_after=5):
        self.Y_range = label_to_range(train_label)
        self.num_classes = len(self.Y_range) - 1
        self.D_range = [self.k*i for i in range(self.num_classes+1)]
        self.Y = Y
        self.D = np.zeros((self.Y.shape[0], self.D_range[-1]))
        self.X = np.zeros((self.D_range[-1], self.Y.shape[1]))
        if self.k0 > 0:
            self.D0 = np.zeros((self.Y.shape[0], self.k0))
            self.X0 = np.zeros((self.k0, self.Y.shape[1]))

        # init
        if verbose:
            print('initializing ... ')
        self._initialize()
        if verbose:
            print('initial loss %.4f' % self.loss())

        # train
        for it in range(iterations):
            # update D
            self._updateD()
            # update D0
            if self.k0 > 0:
                self._updateD0()
            self._updateXX0()
            if verbose and (it == 0 or (it + 1) % show_after == 0):
                print('iter \t%3d/%3d \t loss %.4f' % (it+1, iterations, self.loss()))

    def _updateD(self):
        Y = self.Y if self.k0 == 0 else self.Y - np.dot(self.D0, self.X0)
        F = buildMhat(np.dot(self.X, self.X.T), self.D_range, self.D_range)
        E = np.dot(Y, buildMhat(self.X.T, self.Y_range, self.D_range))
        self.D = ODL_updateD(self.D, E, F)

    def _extract_fromX1(self, X1):
        K = self.D_range[-1]
        return (X1[:K, :], X1[K:, :]) if self.k0 > 0 else (X1[:K, :], _zero)

    def _updateXX0(self):
        clf = _UpdateXX0(self.Y, self.Y_range, self.D, self.D_range, self.D0,
                         self.k0, lambd=self.lambd, lambd2=self.lambd2)

        X1 = np.vstack((self.X, self.X0)) if self.k0 > 0 else self.X
        X1 = clf.solve(Xinit=X1)
        self.X, self.X0 = self._extract_fromX1(X1)

    def _buildYhat(self):
        """
        Yhat = [Yhat_1, Yhat_2, ..., Yhat_C]
        where Yhat_c = Yc - Dc*Xcc
        """
        Yhat = np.zeros_like(self.Y)
        for c in range(self.num_classes):
            Yc = get_block_col(self.Y, c, self.Y_range)
            Dc = get_block_col(self.D, c, self.D_range)
            Xcc = get_block(self.X, c, c, self.D_range, self.Y_range)
            Yhat[:, self.Y_range[c]: self.Y_range[c+1]] = Yc - np.dot(Dc, Xcc)
        return Yhat

    def _updateD0(self):
        Ybar = self.Y - np.dot(self.D, self.X)
        L = (Ybar + self._buildYhat())/2
        self.D0 = min_rank_dict(L, self.X0, self.eta/2, Dinit=self.D0, iterations=50)

    def _initialize(self):
        for c in range(self.num_classes):
            clf = ODL(lambd=self.lambd, k=self.D_range[c+1] - self.D_range[c])
            clf.fit(self._getYc(c), iterations=5)
            self.D[:, self.D_range[c]:self.D_range[c+1]] = clf.D
            self.X[self.D_range[c]: self.D_range[c+1],
                   self.Y_range[c]: self.Y_range[c+1]] = clf.X

        if self.k0 > 0:
            odl = ODL(lambd=self.lambd, k=self.k0)
            odl.fit(self.Y)
            self.D0 = odl.D
            self.X0 = odl.X

    def _fidelity(self):
        """
        Calculating the fidelity term in FDDL
        sum_{c=1}^C ||Y_c - D_cX^c_c||_F^2 + sum_{i != c} ||D_c X^c_i||_F^2$
        """
        cost = 0
        Y = self.Y - np.dot(self.D0, self.X0) if self.k0 > 0 else self.Y.copy()
        for c in range(self.num_classes):
            Yc = get_block_col(Y, c, self.Y_range)
            Dc = get_block_col(self.D, c, self.D_range)
            Xc = get_block_row(self.X, c, self.D_range)
            Xcc = get_block_col(Xc, c, self.Y_range)
            cost += normF2(Yc - np.dot(Dc, Xcc))
            for i in range(self.num_classes):
                if i == c:
                    continue
                Xci = get_block_col(Xc, i, self.Y_range)
                cost += normF2(np.dot(Dc, Xci))
        return cost

    def _discriminative(self):
        """
        * calculating the discriminative term in FDDL[[4]](#fn_fdd):
        * $\|X\|_F^2 + \sum_{c=1}^C (\|Xc - Mc\|_F^2 - \|Mc - M\|_F^2) $
        """
        cost = normF2(self.X)
        m = np.mean(self.X, axis=1)
        for c in range(self.num_classes):
            Xc = get_block_col(self.X, c, self.Y_range)
            Mc = build_mean_matrix(Xc)
            cost += normF2(Xc - Mc)
            M = repmat(m, 1, Xc.shape[1])
            cost -= normF2(Mc - M)
        return cost

    def loss(self):
        Y = self.Y.copy()
        if self.k0 > 0:
            Y -= np.dot(self.D0, self.X0)
        cost = 0.5*normF2(Y - np.dot(self.D, self.X)) + \
            0.5*self._fidelity() + \
            0.5*self.lambd2*self._discriminative() + \
            self.lambd*norm1(self.X)

        if self.k0 > 0:
            cost += self.lambd*norm1(self.X0) + \
                0.5*self.lambd2*normF2(self.X0 - build_mean_matrix(self.X0)) \
                + self.eta*nuclearnorm(self.D0)
        return cost

    def predict(self, Y):
        N = Y.shape[1]
        E = np.zeros((self.num_classes, N))
        for c in range(self.num_classes):
            # Dc in D only
            Dc_ = get_block_col(self.D, c, self.D_range)
            # Dc in D and D0
            Dc = np.hstack((Dc_, self.D0)) if self.k0 > 0 else Dc_
            print("Dc_sum:",Dc_.sum())
            print("sparsity",np.count_nonzero(self.D0))

            lasso = Lasso(Dc, lambd=self.lambd)
            lasso.fit(Y)
            Xc = lasso.solve()
            residual_matrix = Y - np.dot(Dc, Xc)
            E[c, :] = 0.5*np.sum(residual_matrix*residual_matrix, axis=0) +\
                self.lambd*np.sum(np.abs(Xc), axis=0)
        pred = np.argmin(E, axis=0) + 1
        return pred


# def mini_test_unit():
#     print('\n================================================================')
#     print('Mini Unit test: Low-rank shared Dictionary Learning')
#     dataset = 'myFlower'
#     N_train = 5
#     Y_train, Y_test, label_train, label_test = training_test_split(dataset, N_train)
#     lrsdl = LRSDL(lambd=0.01, lambd2=0.01, eta=0.1, k=4, k0=5)
#     lrsdl.fit(Y_train, label_train, iterations=30, verbose=True)
#     lrsdl.evaluate(Y_test, label_test)


# def mini_test_unit_FDDL():
#     print('\n================================================================')
#     print('Mini Unit test: Fisher Disrciminant Dicationary Learning')
#     dataset = 'myYaleB'
#     N_train = 5
#     Y_train, Y_test, label_train, label_test = train_test_split(dataset, N_train)
#     lrsdl = LRSDL(lambd=0.01, lambd2=0.01, eta=0.1, k=4, k0=0)
#     lrsdl.fit(Y_train, label_train, iterations=30, verbose=True)
#     lrsdl.evaluate(Y_test, label_test)


# def test_unit_FDDL():
#     print('\n================================================================')
#     print('Unit test: Fisher Disrciminant Dicationary Learning')
#     dataset = 'myYaleB'
#     N_train = 30
#     Y_train, Y_test, label_train, label_test = train_test_split(dataset, N_train)
#     lrsdl = LRSDL(lambd=0.01, lambd2=0.01, eta=0.1, k=20, k0=0)
#     lrsdl.fit(Y_train, label_train, iterations=30, verbose=True)
#     lrsdl.evaluate(Y_test, label_test)


def test_unit():
    print('\n================================================================')
    print('Unit test: Low-rank shared Dictionary Learning')
    # dataset = 'myFlower'
    # N_train = 60
    # Y_train, Y_test, label_train, label_test = training_test_split(dataset, N_train)
    # lrsdl = LRSDL(lambd=0.01, lambd2=0.01, eta=0.1, k=20, k0=10)
    # lrsdl.fit(Y_train, label_train, iterations=10, verbose=True)
    # lrsdl.evaluate(Y_test, label_test)
    dataset = 'mnist'
    N_train = 100
    Y_train, Y_test, label_train, label_test = training_test_split(dataset, N_train)
    lrsdl = LRSDL(lambd=0.05, lambd2=0.01, eta=0.001, k=20, k0=6)
    lrsdl.fit(Y_train, label_train, iterations=40, verbose=True)
    lrsdl.evaluate(Y_test, label_test)

if __name__ == '__main__':
#     mini_test_unit_FDDL()
  #  mini_test_unit()
#     test_unit_FDDL()
    test_unit()



Unit test: Low-rank shared Dictionary Learning
(60000, 784)
(10000, 784)
label train (60000,)
label train [0. 0. 0. ... 9. 9. 9.]
range [0, 5923, 12665, 18623, 24754, 30596, 36017, 41935, 48200, 54051, 60000]
Y_train [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Label_train [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3

In [ ]:
import scipy.io
mat = scipy.io.loadmat('mnist-original.mat')
mat["data"].shape


(784, 70000)

In [ ]:
import scipy.io
matf = scipy.io.loadmat('myFlower102.mat')
matf["Y_train"].shape


(10000, 1020)

In [ ]:
from google.colab import files
# https://www.kaggle.com/general/74235
!pip install -q kaggle

In [ ]:
files.upload();
! mkdir ~/.kaggle
! mv kaggle.json ~/.kaggle/

KeyboardInterrupt: ignored

In [ ]:
!ls -la
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d avnishnish/mnist-original

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip -q /content/mnist-original.zip

replace mnist-original.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
import matplotlib.pyplot as plt
def plot_images_sample(X, Y):
    # Draw plot for images sample

    plt.figure(figsize=(10,10))
    rand_indicies = np.random.randint(len(X), size=25)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        index = rand_indicies[i]
        plt.imshow(np.squeeze(X[index]), cmap=plt.cm.binary)
        plt.xlabel

In [ ]:
# Draw plot for images sample
dataset = 'mnist'
N_train = 100
Y_train, Y_test, label_train, label_test = training_test_split(dataset, N_train)
print(label_train)

(60000, 784)
(10000, 784)
label train (60000,)
label train [0. 0. 0. ... 9. 9. 9.]
range [0, 5923, 12665, 18623, 24754, 30596, 36017, 41935, 48200, 54051, 60000]
Y_train [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Label_train [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [ ]:
print(label_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 